In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the monthly SST data for 2024 and January 2025
sst_2024 = pd.read_csv('SurfSpot_SST(Monthly Mean 2020).csv')  
sst_jan_2025 = pd.read_csv('2021/SurfSpot_SST(Monthly Mean 2021).csv')  

In [3]:
# Convert 'Month' column to datetime
sst_2024['Month'] = pd.to_datetime(sst_2024['Month'])
sst_jan_2025['Month'] = pd.to_datetime(sst_jan_2025['Month'])

In [4]:
# Initialize an empty list to store results
daily_results = []

In [5]:
# Get the list of unique surf spots
surf_spots = sst_2024['Surf Spot'].unique()

Interpolate SST values

In [7]:
# Iterate over each surf spot
for spot in surf_spots:
    # Filter 2024 data for the current surf spot
    spot_data_2024 = sst_2024[sst_2024['Surf Spot'] == spot].sort_values('Month')

    # Initialize the all_dates and all_values arrays for interpolation
    all_dates = []
    all_values = []

    # Add the data for the first month (January 2023)
    all_dates.append(pd.to_datetime('2020-01-01'))
    all_values.append(spot_data_2024[spot_data_2024['Month'].dt.month == 1]['SST (°C)'].values[0])

    # Iterate through each month in 2023 (Jan-Dec) for interpolation
    for month in range(1, 13):
        # Get the corresponding monthly mean SST for the current month in 2024
        month_data = spot_data_2024[spot_data_2024['Month'].dt.month == month]
        
        if not month_data.empty:
            monthly_sst = month_data['SST (°C)'].values[0]
        else:
            monthly_sst = None
        
        # Add the data for the current month (except January, already added)
        if monthly_sst is not None:
            all_dates.append(pd.to_datetime(f'2020-{month:02d}-01'))
            all_values.append(monthly_sst)

    # Add the data for January 2024 to help with interpolation at the end
    jan_2025_sst = sst_jan_2025[sst_jan_2025['Surf Spot'] == spot]['SST (°C)'].values[0]
    all_dates.append(pd.to_datetime('2021-01-01'))
    all_values.append(jan_2025_sst)

    # Interpolate the daily SST values between all months
    for i in range(1, len(all_dates)):
        # Get the start and end date for the interpolation
        start_date = all_dates[i-1]
        end_date = all_dates[i]

        # Generate daily dates for the current month
        daily_dates = pd.date_range(start=start_date, end=end_date - pd.Timedelta(days=1))

        # Linearly interpolate between the SST values for this interval
        interp_values = np.interp(
            pd.to_datetime(daily_dates).view('int64'),  # Convert dates to numeric format
            [start_date.value, end_date.value],  # Numeric dates for start and end
            [all_values[i-1], all_values[i]]  # Corresponding SST values for the interval
        )

        # Save interpolated daily SST data
        daily_sst_data = pd.DataFrame({
            'Date': daily_dates,
            'Surf Spot': spot,
            'SST (°C)': interp_values
        })

        # Append to the results
        daily_results.append(daily_sst_data)

# Combine all daily results into a single DataFrame
final_results = pd.concat(daily_results)

In [8]:
# Save the final results to a CSV file
final_results.to_csv('SurfSpot_SST(Daily Mean 2020).csv', index=False)

print("Daily SST data successfully generated and saved.")

Daily SST data successfully generated and saved.
